In [1]:
# Install necessary libraries
!pip install tensorflow scikit-learn matplotlib opencv-python

In [ ]:
# Import required libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import cv2
import os

# Load and preprocess dataset (adjust dataset path accordingly)
def load_dataset(base_path, target_size=(128, 128)):
    classes = ['Non_Demented', 'Very_Mild_Demented', 'Mild_Demented', 'Moderate_Demented']
    images, labels = [], []
    for idx, label in enumerate(classes):
        class_path = os.path.join(base_path, label)
        for file in os.listdir(class_path):
            image = cv2.imread(os.path.join(class_path, file))
            if image is not None:
                image = cv2.resize(image, target_size)
                images.append(image)
                labels.append(idx)
    return np.array(images), np.array(labels)

# Assuming the dataset is stored in /content/dataset
base_path = '/content/dataset'
images, labels = load_dataset(base_path)

# Normalize the data
images = images / 255.0

# Split the data into training, validation, and test sets
from sklearn.model_selection import train_test_split
x_train, x_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.3, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

# Prepare input variants
# 1. Original Images
x_train_original = x_train
x_val_original = x_val
x_test_original = x_test

# 2. Lower Resolution Images
def resize_images(images, target_size=(64, 64)):
    resized = [cv2.resize(img, target_size) for img in images]
    return np.array(resized)

x_train_resized = resize_images(x_train, target_size=(64, 64))
x_val_resized = resize_images(x_val, target_size=(64, 64))
x_test_resized = resize_images(x_test, target_size=(64, 64))

# 3. PCA Applied Images
def apply_pca(images, n_components=100):
    flattened_images = images.reshape(images.shape[0], -1)
    pca = PCA(n_components=n_components)
    pca_images = pca.fit_transform(flattened_images)
    return pca_images, pca

x_train_pca, pca = apply_pca(x_train, n_components=100)
x_val_pca = pca.transform(x_val.reshape(x_val.shape[0], -1))
x_test_pca = pca.transform(x_test.reshape(x_test.shape[0], -1))

# Define a simple CNN model
def build_cnn(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(4, activation='softmax')  # 4 classes
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Train and evaluate models
def train_and_evaluate(x_train, x_val, x_test, y_train, y_val, y_test, input_shape):
    model = build_cnn(input_shape)
    model.summary()
    history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10, batch_size=32)
    test_loss, test_acc = model.evaluate(x_test, y_test)
    print(f"Test Accuracy: {test_acc:.4f}")
    return history, test_acc

# Run experiments
print("Training on Original Images...")
history_original, acc_original = train_and_evaluate(x_train_original, x_val_original, x_test_original, y_train, y_val, y_test, input_shape=(128, 128, 3))

print("Training on Lower Resolution Images...")
history_resized, acc_resized = train_and_evaluate(x_train_resized, x_val_resized, x_test_resized, y_train, y_val, y_test, input_shape=(64, 64, 3))

print("Training on PCA Applied Images...")
# PCA data needs reshaping for CNN compatibility
x_train_pca_reshaped = x_train_pca.reshape(x_train_pca.shape[0], 10, 10, 1)
x_val_pca_reshaped = x_val_pca.reshape(x_val_pca.shape[0], 10, 10, 1)
x_test_pca_reshaped = x_test_pca.reshape(x_test_pca.shape[0], 10, 10, 1)

history_pca, acc_pca = train_and_evaluate(x_train_pca_reshaped, x_val_pca_reshaped, x_test_pca_reshaped, y_train, y_val, y_test, input_shape=(10, 10, 1))

# Compare results
print("Experiment Results:")
print(f"Original Images Test Accuracy: {acc_original:.4f}")
print(f"Lower Resolution Images Test Accuracy: {acc_resized:.4f}")
print(f"PCA Applied Images Test Accuracy: {acc_pca:.4f}")
